# 5-11 Process

## 5-11-1 子プロセスの生成

In [1]:
pid = Process.fork do
  puts 'From child process'
  exit
end


15653

In [2]:
pid = Process.fork

if pid
  puts "forked: #{$$}"
else
  puts 'From child process'
  exit
end

forked: 15652


## 5-11-2 子プロセスの待ち合わせ

In [3]:
pid = Process.fork do
  sleep 10
  exit
end

Process.waitpid pid


15655

## 5-11-3 プログラムのデーモン化

current_time.rb として保存し実行

``` ruby
require 'socket'

Process.daemon # プロセスをデーモンとして動作させる

TCPServer.open 'localhost', 4423 do |server|
  loop do
    client = server.accept # クライアントからの接続を待ち受ける

    client.puts Time.now   # 現在時刻を返す

    client.close
  end
end
```

# 5-12 Struct

## 5-12-1 基本的な使い方

In [4]:
Human = Struct.new(:age, :gender)

human = Human.new(10, 'male')
p human.age
p human.gender
human.age = 20

p human[:age]
human[:age] = 100

p Human.members
human.members

10
"male"
20
[:age, :gender]


[:age, :gender]

In [5]:
# 他の言語のように、値渡しになるわけではなく、Struct も Class
human2 = human
human2.age = 50
p human2.age
human.age


Human.class

50


Class

In [6]:
Struct.new('Human', 'age', 'gender')

Struct::Human

In [7]:
p Struct::Human.new
p Human = Struct.new('Age', 'Gender')

p Human = Struct.new('age', 'gender')


#<struct Struct::Human age=nil, gender=nil>


(pry):41: warning: already initialized constant Human
(pry):19: warning: previous definition of Human was here


Struct::Age


NameError: identifier age needs to be constant

In [8]:
Foo = Struct.new(:one, :two, :three)
p foo = Foo.new('a', 'b', 'c')

foo.each {|value| puts value }

p foo.members

foo.each_pair { |field, value| puts field, value }

p foo.map { |value| value.upcase }

Hash[foo.each_pair.to_a]


#<struct Foo one="a", two="b", three="c">
a
b
c
[:one, :two, :three]
one
a
two
b
three
c
["A", "B", "C"]


{:one=>"a", :two=>"b", :three=>"c"}

In [9]:
# Ruby 2.3 から Struct にも dig メソッドが追加
Bar = Struct.new(:a)
p bar = Bar.new({b: [:c, :d]})

bar.dig(:a, :b, 1)

#<struct Bar a={:b=>[:c, :d]}>


:d

## 5-12-2 メソッドの定義

In [10]:
Human = Struct.new('Human', :age, :gender) do
  def teen?
    (13..19).include?(age)
  end
end

p Human.new(10).teen?
p Human.new(14).teen?

Human.ancestors

(pry):61: warning: redefining constant Struct::Human
(pry):61: warning: already initialized constant Human
(pry):41: warning: previous definition of Human was here


false
true


[Struct::Human, Struct, Enumerable, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]

In [11]:
class Human2 < Struct.new('Human', :age, :gender)
  def teen?
    (13..19).include?(age)
  end
end

p Human2.new(10).teen?
p Human2.new(14).teen?

Human2.ancestors

(pry):71: warning: redefining constant Struct::Human


false
true


[Human2, Struct::Human, Struct, Enumerable, Object, PP::ObjectMixin, JSON::Ext::Generator::GeneratorMethods::Object, Kernel, BasicObject]

# 5-13 Marshal

In [12]:
array = [1, '2', [3], Time.now]

p m = Marshal.dump(array)
Marshal.load(m)

"\x04\b[\ti\x06I\"\x062\x06:\x06ET[\x06i\bIu:\tTime\r\xA9\xA3\x1D\x80\xF5\x17o@\a:\voffseti\x02\x90~:\tzoneI\"\bJST\x06;\x00F"


[1, "2", [3], 2018-09-29 18:16:06 +0900]

## 5-13-1 永続化できないオブジェクト

## 5-13-2 バージョン

In [13]:
p Marshal::MAJOR_VERSION
Marshal::MINOR_VERSION

4


8

## 5-13-3 カスタマイズ

In [14]:
class Something
  attr_accessor :source, :temporary
  
  def marshal_dump
    source
  end
  
  def marshal_load(s)
    self.source = s
  end
end

origin = Something.new
origin.source = 'necessary'
origin.temporary = 'drop me'

p data = Marshal.dump(origin)

restored = Marshal.load(data)
p restored.source
p restored.temporary

"\x04\bU:\x0ESomethingI\"\x0Enecessary\x06:\x06ET"
"necessary"
nil


# 5-14 ObjectSpace

## 5-14-1 オブジェクトの操作

In [15]:
ObjectSpace.each_object(Class) {|c| p c}

Gem::Resolver::SpecSpecification
#<Class:FileUtils::Verbose>
Gem::Resolver::Specification
#<Class:Gem::Version>
Gem::Version
Gem::Resolver::Molinillo::Resolver::Resolution::Conflict
#<Class:Gem::Resolver::Molinillo::DependencyGraph::AddEdgeNoCircular>
Gem::Resolver::Molinillo::DependencyGraph::AddEdgeNoCircular
Gem::Resolver::Molinillo::Resolver::Resolution
Gem::UnsatisfiableDependencyError
Gem::SystemExitException
Gem::VerificationError
Gem::RubyVersionMismatch
Gem::RemoteSourceException
Gem::RemoteInstallationSkipped
Gem::RemoteInstallationCancelled
Gem::RemoteError
Gem::OperationNotSupportedError
Gem::InvalidSpecificationException
Gem::RuntimeRequirementNotMetError
Gem::InstallError
Gem::ImpossibleDependenciesError
Gem::SpecificGemNotFoundException
Gem::GemNotFoundException
Gem::FormatException
Gem::FilePermissionError
Gem::EndOfYAMLException
Gem::DocumentError
Gem::GemNotInHomeException
Gem::DependencyResolutionError
Gem::DependencyRemovalException
Gem::DependencyError
Gem::Command

#<Class:Process::UID>
Process::Tms
Process::Status
Process::Waiter
#<Class:Process>
Thread::ConditionVariable
Thread::SizedQueue
ClosedQueueError
Thread::Queue
Thread::Mutex
ThreadError
ThreadGroup
RubyVM::InstructionSequence
Thread::Backtrace::Location
Thread::Backtrace
#<Class:Thread>
Thread
RubyVM
Enumerator::Yielder
Enumerator::Generator
StopIteration
Enumerator::Lazy
Enumerator
ObjectSpace::WeakMap
#<Class:ObjectSpace>
#<Class:GC::Profiler>
#<Class:GC>
#<Class:Math>
Math::DomainError
Binding
UnboundMethod
Method
SystemStackError
LocalJumpError
Proc
Gem::Dependency
#<Class:Signal>
Random
#<Class:#<Class:BasicObject>>
#<Class:#<Class:Object>>
#<Class:#<Class:Module>>
#<Class:#<Class:Class>>
#<Class:Time>
Time
#<Class:Dir>
Dir
File::Stat
#<Class:FileTest>
File
ARGF.class
#<Class:URI::Generic>
URI::Generic
IO::EINPROGRESSWaitWritable
IO::EINPROGRESSWaitReadable
IO::EAGAINWaitWritable
IO::EAGAINWaitReadable
#<Class:IO>
IO
EOFError
IOError
Range
#<Class:Marshal>
Encoding::Converter
Enco

Gem::Resolver::Molinillo::DependencyGraph::SetPayload
#<Class:Pry::Indent>
Pry::Indent
Gem::Resolver::ActivationRequest
Gem::Source::SpecificFile
CZMQ::FFI::Zchunk::DestroyedError
#<Class:CZMQ::FFI::Zchunk>
CZMQ::FFI::Zchunk
Gem::Source::Vendor
Gem::Source::Lock
Gem::Source::Local
#<Class:Gem::BundlerVersionFinder>
CZMQ::FFI::Zcert::DestroyedError
#<Class:CZMQ::FFI::Zcert>
CZMQ::FFI::Zcert
IRuby::OStream
CZMQ::FFI::Zactor::DestroyedError
#<Class:CZMQ::FFI::Zactor>
CZMQ::FFI::Zactor
IRuby::PryBackend
IRuby::PlainBackend
Pry::Command::SaveFile
#<Class:IRuby::HTML>
#<Class:IRuby::LaTeX>
FFI::AutoPointer::CallableReleaser
FFI::AutoPointer::DefaultReleaser
FFI::AutoPointer::Releaser
#<Class:FFI::AutoPointer>
FFI::AutoPointer
Tempfile::Remover
#<Class:Tempfile>
Tempfile
FFI::StructLayoutBuilder
#<Class:#<Class:0x00007ffdd60e9670>>
#<Class:0x00007ffdd60e9670>
#<Class:Dir::Tmpname>
Digest::Base
#<Class:Digest::Class>
Digest::Class
#<Class:Digest>
FFI::NullPointerError
#<Class:FFI::AbstractMemo

CZTop::Monitor
Pry::Command::Cd
CZMQ::FFI::Zloop::DestroyedError
#<Class:CZMQ::FFI::Zloop>
CZMQ::FFI::Zloop
Pry::Editor
#<Class:FFI::Library>
FFI::NotFoundError
CZMQ::FFI::Ztimerset::DestroyedError
#<Class:CZMQ::FFI::Ztimerset>
CZMQ::FFI::Ztimerset
Pry::Command::Exit
Pry::Command::Help
Pry::ObjectPath
#<Class:FFI::StrPtrConverter>
FFI::StrPtrConverter
#<Class:FFI>
#<Class:Pry::Command::Edit::FileAndLineLocator>
#<Class:SecureRandom>
CZMQ::FFI::Zmsg::DestroyedError
#<Class:CZMQ::FFI::Zmsg>
CZMQ::FFI::Zmsg
OpenSSL::Cipher::CipherError
OpenSSL::Cipher
OpenSSL::BN
OpenSSL::BNError
OpenSSL::OpenSSLError
#<Class:OpenSSL>
FFI::StructLayout::Array
FFI::StructLayout::Function
FFI::StructLayout::Pointer
FFI::StructLayout::String
FFI::StructLayout::Number
FFI::StructLayout::Field
FFI::StructLayout
FFI::StructByReference
FFI::StructByValue
#<Class:FFI::Buffer>
FFI::Buffer
FFI::MemoryPointer
FFI::Function
FFI::FunctionType
Pry::Command::FixIndent
CZTop::Message::FramesAccessor
Pry::Command::ImportS

1116

In [16]:
a = [4, 4, 2, 3]
p a.object_id
ObjectSpace._id2ref(a.object_id)

70364089831080


[4, 4, 2, 3]

In [17]:
o = Object.new
ObjectSpace.define_finalizer(o, proc { puts 'finalizing...'})
o = nil
GC.start

In [18]:
ob = Object.new
ObjectSpace.define_finalizer(ob, proc { puts 'finalizing 1...'})
ObjectSpace.define_finalizer(ob, proc { puts 'finalizing 2...'})
ObjectSpace.define_finalizer(ob, proc { raise 'finalizing 1...'})
ob = nil
GC.start

finalizing...
